In [8]:
import uuid
import requests
from qdrant_client import QdrantClient, models

In [5]:
qd_client = QdrantClient("http://localhost:6333")
qd_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='zoomcamp-rag'), CollectionDescription(name='zoomcamp-faq')])

In [6]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


In [7]:
qd_client.create_collection(
    collection_name="zoomcamp-sparse",
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [13]:
qd_client.upsert(
    collection_name="zoomcamp-sparse",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc['text'],
                    model="Qdrant/bm25"
                ),
            },
            payload={
                'text': doc['text'],
                'section': doc['section'],
                'course': course['course']
            }
        )
        for course in documents_raw
        for doc in course['documents']
    ]

)

Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 54.47it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)